In [1]:
import numpy as np
import pandas as pd

import xgboost as xgb

import optuna

import warnings
warnings.filterwarnings("ignore")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def custom_score(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred)/y_pred)

In [3]:
feats = ['lon', 'lat', 'hour', 'is_working_day', 'is_holiday', 'temp', 'is_day', 'day', 'weekday',
 'month', 'year', 'is_night', 'is_evening']
target = 'count'

In [4]:
df = pd.read_csv('/home/ubuntu/train_data4.csv')
df_v = pd.read_csv('/home/ubuntu/val_data3.csv')

In [13]:
def objective(trial, X_train=df[feats], y_train=df[target], X_eval=df_v[feats], y_eval=df_v[target]):
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-2, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-2, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', np.logspace(-3, 0, 10)),
        'n_estimators': 1000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [42]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_eval)
    y_pred = np.array([k if k > 0 else 1 for k in y_pred])
    score = np.mean(np.abs(y_eval - y_pred)/y_pred)
    return score

In [15]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-10-04 07:26:06,763] A new study created in memory with name: no-name-c280b41e-35c1-4111-a2c5-c366f348d15c
[I 2023-10-04 07:26:39,629] Trial 0 finished with value: 1.0537787762531503 and parameters: {'lambda': 0.12243613494634178, 'alpha': 0.19787103248004628, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 67}. Best is trial 0 with value: 1.0537787762531503.
[I 2023-10-04 07:27:34,188] Trial 1 finished with value: 1.1868381124109517 and parameters: {'lambda': 0.5872501806715846, 'alpha': 0.22469455079278272, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.4641588833612777, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 177}. Best is trial 0 with value: 1.0537787762531503.
[I 2023-10-04 07:28:01,197] Trial 2 finished with value: 1.7115517394153568 and parameters: {'lambda': 0.02218887868308032, 'alpha': 0.06398461016425631, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate':

[I 2023-10-04 07:43:18,162] Trial 24 finished with value: 0.9493301213892332 and parameters: {'lambda': 3.8207218463586354, 'alpha': 9.377866846752314, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.046415888336127774, 'max_depth': 13, 'random_state': 42, 'min_child_weight': 87}. Best is trial 23 with value: 0.9487430070584908.
[I 2023-10-04 07:44:05,640] Trial 25 finished with value: 0.9448560835934702 and parameters: {'lambda': 5.125338326728199, 'alpha': 8.715584727679076, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.046415888336127774, 'max_depth': 13, 'random_state': 42, 'min_child_weight': 72}. Best is trial 25 with value: 0.9448560835934702.
[I 2023-10-04 07:44:54,647] Trial 26 finished with value: 0.9324790491376151 and parameters: {'lambda': 4.191095833706131, 'alpha': 9.578659716094988, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.046415888336127774, 'max_depth': 13, 'random_state': 42, 'min_child_weight': 64}. Best is trial 26

[I 2023-10-04 07:55:22,178] Trial 48 finished with value: 2.490257042689627 and parameters: {'lambda': 1.1070806994805287, 'alpha': 0.5075923417626655, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.001, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 117}. Best is trial 45 with value: 0.8599676622528458.
[I 2023-10-04 07:56:03,039] Trial 49 finished with value: 1.0575554473569238 and parameters: {'lambda': 0.3109996858547079, 'alpha': 0.2280823252395907, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.004641588833612777, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 99}. Best is trial 45 with value: 0.8599676622528458.
[I 2023-10-04 07:56:26,024] Trial 50 finished with value: 0.8692128964237918 and parameters: {'lambda': 0.8007275507000852, 'alpha': 0.6101438403269831, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.09999999999999999, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 139}. Best is trial 45 with value

[I 2023-10-04 08:07:16,881] Trial 72 finished with value: 0.8652215224101525 and parameters: {'lambda': 0.6194651055105677, 'alpha': 0.8236399573440112, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.09999999999999999, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 118}. Best is trial 57 with value: 0.8292225722197529.
[I 2023-10-04 08:07:41,031] Trial 73 finished with value: 0.8642617698875997 and parameters: {'lambda': 0.6231651451275303, 'alpha': 0.8869651530752415, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.09999999999999999, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 128}. Best is trial 57 with value: 0.8292225722197529.
[I 2023-10-04 08:08:05,443] Trial 74 finished with value: 2.5550568460209777 and parameters: {'lambda': 0.6325808058128709, 'alpha': 1.3785877275504022, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 1.0, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 130}. Best is trial 57 with value: 0

[I 2023-10-04 08:22:19,200] Trial 96 finished with value: 0.8459910211253199 and parameters: {'lambda': 1.6677778136926587, 'alpha': 0.6345228151419143, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.09999999999999999, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 22}. Best is trial 57 with value: 0.8292225722197529.
[I 2023-10-04 08:23:01,139] Trial 97 finished with value: 1.1474473031540178 and parameters: {'lambda': 1.5960989279862037, 'alpha': 0.6519917708909307, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.4641588833612777, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 23}. Best is trial 57 with value: 0.8292225722197529.
[I 2023-10-04 08:23:43,686] Trial 98 finished with value: 62.978929441053836 and parameters: {'lambda': 3.0129761300111664, 'alpha': 0.47562465933178294, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 1.0, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 3}. Best is trial 57 with value: 0.

[I 2023-10-04 08:37:04,837] Trial 120 finished with value: 1.2247528851662002 and parameters: {'lambda': 0.7200803213769044, 'alpha': 0.33948385364274386, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.004641588833612777, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 5}. Best is trial 116 with value: 0.8212790385228022.
[I 2023-10-04 08:37:34,783] Trial 121 finished with value: 0.8347526671097893 and parameters: {'lambda': 1.2507175133617008, 'alpha': 0.6063647309196004, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 23}. Best is trial 116 with value: 0.8212790385228022.
[I 2023-10-04 08:38:04,128] Trial 122 finished with value: 0.8190365942219976 and parameters: {'lambda': 1.4856519362340475, 'alpha': 0.4471117097112199, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 11}. Best is tri

[I 2023-10-04 08:49:49,724] Trial 144 finished with value: 0.8743255769530459 and parameters: {'lambda': 0.9571613537847685, 'alpha': 0.2211847743931313, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 33}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 08:50:21,985] Trial 145 finished with value: 0.8965783744798902 and parameters: {'lambda': 1.5533761959147963, 'alpha': 0.32561627254351966, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 21}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 08:50:53,191] Trial 146 finished with value: 0.8519095033769759 and parameters: {'lambda': 0.8256937988685176, 'alpha': 0.4896121001148398, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 17}. Best is tri

[I 2023-10-04 09:05:27,519] Trial 168 finished with value: 0.9443084779343731 and parameters: {'lambda': 2.4356819278764674, 'alpha': 0.4744618300805641, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.4641588833612777, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 8}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 09:06:07,455] Trial 169 finished with value: 1.588700486423728 and parameters: {'lambda': 3.329522146478841, 'alpha': 0.46385731609203384, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 1.0, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 14}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 09:06:36,356] Trial 170 finished with value: 0.8404489411586619 and parameters: {'lambda': 2.771020067521873, 'alpha': 0.5332014516230598, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 8}. Best is trial 122 with value: 0.8

[I 2023-10-04 09:18:19,300] Trial 192 finished with value: 0.830507842428841 and parameters: {'lambda': 1.7694349959190574, 'alpha': 0.6197605194282646, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 12}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 09:18:54,002] Trial 193 finished with value: 0.8457361237596868 and parameters: {'lambda': 1.8846898860915302, 'alpha': 0.6191131578261678, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 13}. Best is trial 122 with value: 0.8190365942219976.
[I 2023-10-04 09:19:24,810] Trial 194 finished with value: 0.8276401807510391 and parameters: {'lambda': 1.8965865231077232, 'alpha': 0.38126190787262504, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 1}. Best is trial

Number of finished trials: 200
Best trial: {'lambda': 1.4856519362340475, 'alpha': 0.4471117097112199, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 11}


In [5]:
def objective(trial, X_train=df[feats], y_train=df[target], X_eval=df_v[feats], y_eval=df_v[target]):
    param = {
        'lambda': trial.suggest_categorical('lambda', np.logspace(-1, 1, 10)),
        'alpha': trial.suggest_categorical('alpha', np.logspace(-1, 1, 10)),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', np.logspace(-3, 0, 10)),
        'n_estimators': trial.suggest_categorical('n_estimators', [500, 800, 1000, 1200]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [42]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_eval)
    score = np.mean(np.abs(y_eval - y_pred)/y_pred)
    if score < 0: score = 1e+3
    return score

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-10-04 10:05:21,723] A new study created in memory with name: no-name-320a812b-720a-4195-89cd-4f0dcb6c04e8
[I 2023-10-04 10:05:49,004] Trial 0 finished with value: 1.956895015555346 and parameters: {'lambda': 3.5938136638046254, 'alpha': 10.0, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.0021544346900318847, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42, 'min_child_weight': 216}. Best is trial 0 with value: 1.956895015555346.
[I 2023-10-04 10:06:11,306] Trial 1 finished with value: 1.1825591181268147 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.004641588833612777, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 259}. Best is trial 1 with value: 1.1825591181268147.
[I 2023-10-04 10:06:25,391] Trial 2 finished with value: 1.8058108261839951 and parameters: {'lambda': 3.5938136638046254, 'alpha': 10.0, 'colsample_bytree': 0.7, 'subsa

[I 2023-10-04 10:19:36,483] Trial 23 finished with value: 0.8588211456767235 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 191}. Best is trial 11 with value: 0.8127387280775994.
[I 2023-10-04 10:20:23,561] Trial 24 finished with value: 0.8589896870289754 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 180}. Best is trial 11 with value: 0.8127387280775994.
[I 2023-10-04 10:21:13,211] Trial 25 finished with value: 0.8540574061193957 and parameters: {'lambda': 2.1544346900318834, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.01, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight

[I 2023-10-04 10:32:20,332] Trial 47 finished with value: 1.4341307532142005 and parameters: {'lambda': 3.5938136638046254, 'alpha': 5.994842503189409, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.0021544346900318847, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 288}. Best is trial 30 with value: 0.3566126816441526.
[I 2023-10-04 10:32:38,651] Trial 48 finished with value: 0.5332236346833868 and parameters: {'lambda': 10.0, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 259}. Best is trial 30 with value: 0.3566126816441526.
[I 2023-10-04 10:32:56,864] Trial 49 finished with value: 3.0564377124537887 and parameters: {'lambda': 10.0, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 299}. Best 

[I 2023-10-04 10:44:16,604] Trial 71 finished with value: 1.5166862926019822 and parameters: {'lambda': 3.5938136638046254, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 230}. Best is trial 62 with value: 0.2787538071740227.
[I 2023-10-04 10:44:46,340] Trial 72 finished with value: 1.4986443544842079 and parameters: {'lambda': 3.5938136638046254, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 225}. Best is trial 62 with value: 0.2787538071740227.
[I 2023-10-04 10:45:22,278] Trial 73 finished with value: 1000.0 and parameters: {'lambda': 3.5938136638046254, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 237}. Best 

[I 2023-10-04 11:00:26,020] Trial 95 finished with value: 0.04597498119696537 and parameters: {'lambda': 10.0, 'alpha': 5.994842503189409, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 294}. Best is trial 95 with value: 0.04597498119696537.
[I 2023-10-04 11:01:09,096] Trial 96 finished with value: 1.3911470545704387 and parameters: {'lambda': 10.0, 'alpha': 5.994842503189409, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 253}. Best is trial 95 with value: 0.04597498119696537.
[I 2023-10-04 11:01:52,805] Trial 97 finished with value: 1.3452237796274054 and parameters: {'lambda': 10.0, 'alpha': 5.994842503189409, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42, 'min_child_weight': 274}. Best is trial 95 with value: 0.0459749

...

[I 2023-10-04 13:44:43,194] Trial 334 finished with value: 1.7184795828393054 and parameters: {'lambda': 10.0, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 1000, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 264}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 13:45:36,976] Trial 335 finished with value: 0.8851997166910048 and parameters: {'lambda': 10.0, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 250}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 13:45:57,215] Trial 336 finished with value: 0.9832232446497494 and parameters: {'lambda': 0.46415888336127786, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.4641588833612777, 'n_estimators': 800, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 300}

[I 2023-10-04 14:01:51,821] Trial 358 finished with value: 0.5516588488517338 and parameters: {'lambda': 5.994842503189409, 'alpha': 0.7742636826811269, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 264}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:02:40,033] Trial 359 finished with value: 2.222105439613919 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 251}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:03:27,430] Trial 360 finished with value: 2.5213627446003146 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 270}. Best

[I 2023-10-04 14:17:24,490] Trial 382 finished with value: 1.1100919995326926 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 253}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:17:38,161] Trial 383 finished with value: 2.61171446636117 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.001, 'n_estimators': 500, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 261}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:18:15,414] Trial 384 finished with value: 0.36286785675367844 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.2782559402207124, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 1000, 'max_depth': 11, 'random_state': 42, 'min_child_w

[I 2023-10-04 14:33:07,950] Trial 406 finished with value: 1.486339575944122 and parameters: {'lambda': 3.5938136638046254, 'alpha': 1.2915496650148839, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 1.0, 'n_estimators': 1000, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 251}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:33:57,033] Trial 407 finished with value: 1.2050547328870262 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 1.0, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 263}. Best is trial 122 with value: 0.04429821032394422.
[I 2023-10-04 14:34:43,319] Trial 408 finished with value: 1.7706312913140476 and parameters: {'lambda': 3.5938136638046254, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.0021544346900318847, 'n_estimators': 800, 'max_depth': 17, 'random_state': 

KeyboardInterrupt: 

In [ ]:
# [I 2023-10-04 11:18:35,738] Trial 122 finished with value: 0.04429821032394422 and parameters: 
# {'lambda': 10.0, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.9, 'subsample': 0.6, 
#  'learning_rate': 0.4641588833612777, 'n_estimators': 800, 'max_depth': 17, 'random_state': 42, 'min_child_weight': 264}. 
# Best is trial 122 with value: 0.04429821032394422.

In [10]:
def objective(trial, X_train=df[feats], y_train=df[target], X_eval=df_v[feats], y_eval=df_v[target]):
    param = {
        'lambda': trial.suggest_categorical('lambda', np.logspace(-1, 1, 10)),
        'alpha': trial.suggest_categorical('alpha', np.logspace(-1, 1, 10)),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', np.logspace(-1, 0, 10)),
        'n_estimators': trial.suggest_categorical('n_estimators', [500, 800, 1000, 1200]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [42]),
    }
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_eval)
    score = np.mean(np.abs(y_eval - y_pred)/y_pred)
    if score < 0: score = 2
    return score

In [11]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-10-04 14:43:30,634] A new study created in memory with name: no-name-93d109da-28c1-4fb8-b026-e1b3f06884d0
[I 2023-10-04 14:44:37,642] Trial 0 finished with value: 1.1575058184434903 and parameters: {'lambda': 3.5938136638046254, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.46415888336127786, 'n_estimators': 500, 'max_depth': 17, 'random_state': 42}. Best is trial 0 with value: 1.1575058184434903.
[I 2023-10-04 14:45:00,673] Trial 1 finished with value: 1.3883177335415375 and parameters: {'lambda': 2.1544346900318834, 'alpha': 0.2782559402207124, 'colsample_bytree': 0.3, 'subsample': 0.5, 'learning_rate': 1.0, 'n_estimators': 1200, 'max_depth': 5, 'random_state': 42}. Best is trial 0 with value: 1.1575058184434903.
[I 2023-10-04 14:46:10,414] Trial 2 finished with value: 1.5100014181883612 and parameters: {'lambda': 5.994842503189409, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.359381366

[I 2023-10-04 15:01:48,366] Trial 25 finished with value: 1.2399368631720025 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.09999999999999999, 'n_estimators': 800, 'max_depth': 5, 'random_state': 42}. Best is trial 8 with value: 0.6299678248414955.
[I 2023-10-04 15:02:08,485] Trial 26 finished with value: 1.7998965170974104 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.7742636826811269, 'n_estimators': 500, 'max_depth': 11, 'random_state': 42}. Best is trial 8 with value: 0.6299678248414955.
[I 2023-10-04 15:02:36,715] Trial 27 finished with value: 2.2984945524067966 and parameters: {'lambda': 2.1544346900318834, 'alpha': 5.994842503189409, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.5994842503189408, 'n_estimators': 1000, 'max_depth': 9, 'random_state': 42}. Best is trial 8 with value:

[I 2023-10-04 15:20:24,524] Trial 50 finished with value: 2.0 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.7742636826811269, 'n_estimators': 1200, 'max_depth': 15, 'random_state': 42}. Best is trial 8 with value: 0.6299678248414955.
[I 2023-10-04 15:20:49,843] Trial 51 finished with value: 0.8476490211803271 and parameters: {'lambda': 10.0, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.5994842503189408, 'n_estimators': 800, 'max_depth': 5, 'random_state': 42}. Best is trial 8 with value: 0.6299678248414955.
[I 2023-10-04 15:21:05,676] Trial 52 finished with value: 0.8476490211803271 and parameters: {'lambda': 10.0, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.5994842503189408, 'n_estimators': 800, 'max_depth': 5, 'random_state': 42}. Best is trial 8 with value: 0.6299678248414955.
[I 2023-10-04 15:21:2

[I 2023-10-04 15:34:21,635] Trial 75 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:34:44,684] Trial 76 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:35:11,783] Trial 77 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04

[I 2023-10-04 15:44:08,894] Trial 100 finished with value: 1.2226332935447077 and parameters: {'lambda': 0.09999999999999999, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.35938136638046275, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:44:32,167] Trial 101 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:45:00,530] Trial 102 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169

[I 2023-10-04 15:55:02,090] Trial 125 finished with value: 1.024925366030664 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.4, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:55:26,085] Trial 126 finished with value: 1.2811334111309696 and parameters: {'lambda': 5.994842503189409, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.35938136638046275, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 15:55:56,395] Trial 127 finished with value: 0.906206267215736 and parameters: {'lambda': 0.46415888336127786, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008

[I 2023-10-04 16:05:10,734] Trial 150 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 16:05:34,346] Trial 151 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10-04 16:05:57,646] Trial 152 finished with value: 0.19008497444169623 and parameters: {'lambda': 10.0, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1291549665014884, 'n_estimators': 500, 'max_depth': 13, 'random_state': 42}. Best is trial 70 with value: 0.19008497444169623.
[I 2023-10

KeyboardInterrupt: 

In [17]:
def objective(trial, X_train=df[feats], y_train=df[target], X_eval=df_v[feats], y_eval=df_v[target]):
    param = {
        'lambda': trial.suggest_categorical('lambda', np.logspace(-1, 1, 10)),
        'alpha': trial.suggest_categorical('alpha', np.logspace(-1, 1, 10)),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', np.arange(0.4, 1.05, 0.05)),
        'n_estimators': trial.suggest_categorical('n_estimators', [800]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15]),
        'random_state': trial.suggest_categorical('random_state', [42]),
    }
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_eval)
    score = np.mean(np.abs(y_eval - y_pred)/y_pred)
    if score < 0: score = 2
    return score

In [18]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-10-04 16:18:32,476] A new study created in memory with name: no-name-c0bff39d-a33f-4fd6-b502-eb69c4f38732
[I 2023-10-04 16:19:07,463] Trial 0 finished with value: 2.1042870910365696 and parameters: {'lambda': 10.0, 'alpha': 0.2782559402207124, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.9999999999999999, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42}. Best is trial 0 with value: 2.1042870910365696.
[I 2023-10-04 16:19:57,020] Trial 1 finished with value: 0.13705165679003126 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 13, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:20:16,430] Trial 2 finished with value: 0.9055237082830038 and parameters: {'lambda': 1.2915496650148839, 'alpha': 0.16681005372000587, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.7, 

[I 2023-10-04 16:31:13,868] Trial 25 finished with value: 0.9078462855493701 and parameters: {'lambda': 0.2782559402207124, 'alpha': 1.2915496650148839, 'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 0.55, 'n_estimators': 800, 'max_depth': 7, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:32:37,350] Trial 26 finished with value: 1.766850263275496 and parameters: {'lambda': 5.994842503189409, 'alpha': 0.09999999999999999, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.7999999999999999, 'n_estimators': 800, 'max_depth': 15, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:33:10,374] Trial 27 finished with value: 0.7874153494301852 and parameters: {'lambda': 0.16681005372000587, 'alpha': 2.1544346900318834, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.6499999999999999, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42}. Best is trial 1 with value: 0.13705165679

[I 2023-10-04 16:45:35,408] Trial 50 finished with value: 0.4485048194613693 and parameters: {'lambda': 0.7742636826811269, 'alpha': 1.2915496650148839, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.45, 'n_estimators': 800, 'max_depth': 7, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:45:57,829] Trial 51 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:46:19,475] Trial 52 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.1370516567900

[I 2023-10-04 16:59:22,955] Trial 75 finished with value: 2.0 and parameters: {'lambda': 0.16681005372000587, 'alpha': 0.2782559402207124, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.7, 'n_estimators': 800, 'max_depth': 11, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 16:59:56,134] Trial 76 finished with value: 1.1203940836189128 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.6, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:00:22,427] Trial 77 finished with value: 0.9695043876523237 and parameters: {'lambda': 0.09999999999999999, 'alpha': 0.7742636826811269, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.45, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:01:02,595] Trial 78

[I 2023-10-04 17:10:31,690] Trial 100 finished with value: 0.9526855877635076 and parameters: {'lambda': 0.09999999999999999, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.6, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:10:53,855] Trial 101 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:11:16,555] Trial 102 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.1370516567

...

[I 2023-10-04 17:53:05,592] Trial 200 finished with value: 1.2067281474301403 and parameters: {'lambda': 0.7742636826811269, 'alpha': 0.2782559402207124, 'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 5, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:53:27,621] Trial 201 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 17:53:49,621] Trial 202 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value

[I 2023-10-04 18:03:14,670] Trial 225 finished with value: 1.638305132066525 and parameters: {'lambda': 0.2782559402207124, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:03:38,024] Trial 226 finished with value: 2.0 and parameters: {'lambda': 3.5938136638046254, 'alpha': 0.46415888336127786, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:04:03,874] Trial 227 finished with value: 0.8406325796737762 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.45, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 202

[I 2023-10-04 18:13:31,360] Trial 250 finished with value: 0.6484501857796774 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.9999999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:13:53,088] Trial 251 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:14:17,012] Trial 252 finished with value: 0.5618699979129985 and parameters: {'lambda': 0.09999999999999999, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with valu

[I 2023-10-04 18:24:26,567] Trial 275 finished with value: 0.8717366270607292 and parameters: {'lambda': 0.7742636826811269, 'alpha': 1.2915496650148839, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.4, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:24:48,386] Trial 276 finished with value: 0.2873307302315502 and parameters: {'lambda': 0.7742636826811269, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679003126.
[I 2023-10-04 18:25:10,376] Trial 277 finished with value: 1.1592914244355466 and parameters: {'lambda': 2.1544346900318834, 'alpha': 3.5938136638046254, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.8499999999999999, 'n_estimators': 800, 'max_depth': 9, 'random_state': 42}. Best is trial 1 with value: 0.13705165679

KeyboardInterrupt: 